In [2]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Update the table
cursor.execute("UPDATE travel_logistic SET truck_viable = 0 WHERE truck_viable IS NULL")

# Commit the changes and close the connection
conn.commit()
conn.close()

In [5]:
# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Update the table
cursor.execute("""
    UPDATE travel_logistic
    SET truck_viable = 0
    WHERE truck_distance_km IS NOT NULL
    AND NOT (from_id IN (SELECT id FROM fone_geography WHERE continent = 'Europe') 
             AND to_id IN (SELECT id FROM fone_geography WHERE continent = 'Europe'))
    AND NOT (from_id IN (SELECT id FROM fone_geography WHERE continent = 'North America') 
             AND to_id IN (SELECT id FROM fone_geography WHERE continent = 'North America'))
    AND NOT (from_id IN (SELECT id FROM fone_geography WHERE continent = 'South America') 
             AND to_id IN (SELECT id FROM fone_geography WHERE continent = 'South America'))
    AND truck_distance_km > 2200""")

# Commit the changes and close the connection
conn.commit()
conn.close()

In [4]:
# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Update the table
cursor.execute("""
    UPDATE travel_logistic
    SET truck_viable = 1
    WHERE from_id = (SELECT id FROM fone_geography WHERE circuit_x = 'Silverstone')
       OR to_id = (SELECT id FROM fone_geography WHERE circuit_x = 'Silverstone');
""")

# Commit the changes and close the connection
conn.commit()
conn.close()

Silverstone <-> Mogyoród (Hungaroring): ~1884 km
Silverstone <-> Monza (Monza Circuit): ~1331 km
Silverstone <-> Montmeló (Circuit de Barcelona-Catalunya): ~1612 km
Silverstone <-> Spielberg (Red Bull Ring): ~1587 km
Silverstone <-> Hockenheim (Hockenheimring): ~982 km
Silverstone <-> Imola (Imola Circuit): ~1591 km
Silverstone <-> Magny-Cours (Circuit de Nevers Magny-Cours): ~854 km
Silverstone <-> Stavelot (Spa-Francorchamps): ~652 km
Silverstone <-> Zandvoort (Circuit Zandvoort): ~685 km
Silverstone <-> Valencia (Circuit Ricardo Tormo): ~1987 km
Silverstone <-> Le Castellet (Circuit Paul Ricard): ~1418 km
Silverstone <-> Monte Carlo (Circuit de Monaco): ~1558 km
Silverstone <-> Portimão (Algarve International Circuit): ~2544 km
Silverstone <-> Nürburg (Nürburgring): ~782 km
Silverstone <-> Scarperia e San Piero (Mugello Circuit): ~1623 km
Silverstone <-> Oslo (Norway): ~1875 km 

In [ ]:
# Ensure the database connection is properly managed
try:
    # Fill truck_distance_km for Silverstone routes
    distances = [
        ("Silverstone", "Mogyoród", 1884),
        ("Silverstone", "Monza", 1331),
        ("Silverstone", "Montmeló", 1612),
        ("Silverstone", "Spielberg", 1587),
        ("Silverstone", "Hockenheim", 982),
        ("Silverstone", "Imola", 1591),
        ("Silverstone", "Magny-Cours", 854),
        ("Silverstone", "Stavelot", 652),
        ("Silverstone", "Zandvoort", 685),
        ("Silverstone", "Valencia", 1987),
        ("Silverstone", "Le Castellet", 1418),
        ("Silverstone", "Monte Carlo", 1558),
        ("Silverstone", "Portimão", 2544),
        ("Silverstone", "Nürburg", 782),
        ("Silverstone", "Scarperia e San Piero", 1623),
        ("Silverstone", "Oslo", 1875)
    ]
    # Connect to the database
    conn = sqlite3.connect('planet_fone.db')
    cursor = conn.cursor()
    for from_circuit, to_circuit, distance in distances:
        cursor.execute("""
            UPDATE travel_logistic
            SET truck_distance_km = ?, truck_viable = 1
            WHERE from_id = (SELECT id FROM fone_geography WHERE city_x = ?)
              AND to_id = (SELECT id FROM fone_geography WHERE city_x = ?)
        """, (distance, from_circuit, to_circuit))
        
        cursor.execute("""
            UPDATE travel_logistic
            SET truck_distance_km = ?, truck_viable = 1
            WHERE from_id = (SELECT id FROM fone_geography WHERE city_x = ?)
              AND to_id = (SELECT id FROM fone_geography WHERE city_x = ?)
        """, (distance, to_circuit, from_circuit))



    # Set truck_viable to 0 for all routes from and to Sochi
    cursor.execute("""
        UPDATE travel_logistic
        SET truck_viable = 0
        WHERE from_id = (SELECT id FROM fone_geography WHERE city_x = 'Sochi')
           OR to_id = (SELECT id FROM fone_geography WHERE city_x = 'Sochi')
    """)

    # Commit the changes to the database
    conn.commit()

except sqlite3.OperationalError as e:
    print(f"An error occurred: {e}")
finally:
    # Ensure the connection is closed properly
    if conn:
        conn.close()

An error occurred: database is locked


In [ ]:
# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Define the list of valid cities
valid_cities = ('Silverstone', 'Mogyoród', 'Monza', 'Montmeló', 'Spielberg', 
                'Hockenheim', 'Imola', 'Magny-Cours', 'Stavelot', 'Zandvoort', 
                'Valencia', 'Le Castellet', 'Monte Carlo', 'Portimão', 'Nürburg', 
                'Scarperia e San Piero', 'Oslo')

# Update truck_viable = 0 for routes where the other city is not in the list of valid destinations
cursor.execute(f"""
    UPDATE travel_logistic
    SET truck_viable = 0
    WHERE from_id = (SELECT id FROM fone_geography WHERE city_x = ?)
    AND to_id NOT IN (
        SELECT id FROM fone_geography WHERE city_x IN ({','.join(['?'] * len(valid_cities))})
    )
""", (from_circuit, *valid_cities))

cursor.execute(f"""
    UPDATE travel_logistic
    SET truck_viable = 0
    WHERE to_id = (SELECT id FROM fone_geography WHERE city_x = ?)
    AND from_id NOT IN (
        SELECT id FROM fone_geography WHERE city_x IN ({','.join(['?'] * len(valid_cities))})
    )
""", (from_circuit, *valid_cities))

# Commit the changes to the database
conn.commit()

[2019-2022 Estimates]
🚛 Truck Emissions (per tonne-km)
From sources:

Diesel trucks: ~90 g/tonne-km

✈️ Air Freight Emissions (per tonne-km)
From sources:

Conventional Jet Fuel: ~1000 g/tonne-km

F1 typically moves ~1,000 tonnes of equipment.

Total CO₂ (kg) = Distance (km) × Cargo Mass (tonnes) × Emission Factor (g/tonne-km) ÷ 1000

truck_emissions = truck_distance_km x 1000 x 1000 / 1000
air_emissions = distance_km x 1000 x 90 / 1000

In [15]:
# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()
# Add the effort_score column and populate it based on the conditions
#cursor.execute("""    ALTER TABLE travel_logistic ADD COLUMN effort_score REAL;""")

cursor.execute("""
    UPDATE travel_logistic
    SET effort_score = 
        CASE 
            WHEN truck_viable = 1 THEN truck_distance_km * 0.09
            ELSE distance_km * 1.0
        END;
""")

# Commit the changes to the database
conn.commit()

In [ ]:
# Open the connection if it's not already open
if conn is None or cursor is None:
    conn = sqlite3.connect('planet_fone.db')
    cursor = conn.cursor()

# Add the outbound_year_emissions_t column and populate it based on the year
cursor.execute("""
    ALTER TABLE fone_calendar ADD COLUMN outbound_year_emissions_t FLOAT;
""")

cursor.execute("""
    UPDATE fone_calendar
    SET outbound_year_emissions_t = 
        CASE 
            WHEN year = 2022 THEN 109285
            WHEN year = 2019 THEN 115447
        END
    WHERE year IN (2022, 2019);
""")

# Commit the changes to the database
conn.commit()

In [17]:
# Open the connection if it's not already open
if conn is None or cursor is None:
    conn = sqlite3.connect('planet_fone.db')
    cursor = conn.cursor()

# Add the effort_score column to fone_calendar if it doesn't already exist
#cursor.execute("""    ALTER TABLE fone_calendar ADD COLUMN outbound_effort_score FLOAT;""")

# Update the effort_score column based on the conditions
cursor.execute("""
    UPDATE fone_calendar
    SET outbound_effort_score = (
        SELECT tl.effort_score
        FROM travel_logistic tl
        WHERE tl.id = fone_calendar.outbound_route
          AND fone_calendar.outbound_route IS NOT NULL
    )
    WHERE fone_calendar.year IN (2022, 2019)
      AND fone_calendar.outbound_route IS NOT NULL;
""")

# Commit the changes to the database
conn.commit()

In [ ]:
# Open the connection if it's not already open
if conn is None or cursor is None:
    conn = sqlite3.connect('planet_fone.db')
    cursor = conn.cursor()

# Add the yearly_outbound_effort_score column to fone_calendar if it doesn't already exist
cursor.execute("""
    ALTER TABLE fone_calendar ADD COLUMN yearly_outbound_effort_score FLOAT;
""")

# Update the yearly_outbound_effort_score column by summing outbound_effort_score for each year
cursor.execute("""
    UPDATE fone_calendar
    SET yearly_outbound_effort_score = (
        SELECT SUM(fc2.outbound_effort_score)
        FROM fone_calendar fc2
        WHERE fc2.year = fone_calendar.year
    )
    WHERE fone_calendar.outbound_effort_score IS NOT NULL;
""")

# Commit the changes to the database
conn.commit()

In [ ]:
# Open the connection if it's not already open
if conn is None or cursor is None:
    conn = sqlite3.connect('planet_fone.db')
    cursor = conn.cursor()

# Add the leg_emissions column to fone_calendar if it doesn't already exist
#cursor.execute("""
#    ALTER TABLE fone_calendar ADD COLUMN leg_emissions FLOAT;
#""")

# Update the leg_emissions column based on the given formula
cursor.execute("""
    UPDATE fone_calendar
    SET leg_emissions = outbound_year_emissions_t * 
                        (outbound_effort_score / yearly_outbound_effort_score)
    WHERE outbound_year_emissions_t IS NOT NULL
      AND outbound_effort_score IS NOT NULL
      AND yearly_outbound_effort_score IS NOT NULL;
""")

# Commit the changes to the database
conn.commit()

In [ ]:
# Add the Time_Gap_Days column to fone_calendar if it doesn't already exist
#cursor.execute("""
#    ALTER TABLE fone_calendar ADD COLUMN Time_Gap_Days INTEGER;
#""")

# Update the Time_Gap_Days column by calculating the difference in days to the next date in the same year
cursor.execute("""
    UPDATE fone_calendar
    SET Time_Gap_Days = (
        SELECT julianday(fc2.date) - julianday(fc1.date)
        FROM fone_calendar fc1
        JOIN fone_calendar fc2
        ON fc1.year = fc2.year AND fc1.date < fc2.date
        WHERE fone_calendar.id = fc1.id
        ORDER BY fc2.date ASC
        LIMIT 1
    )
    WHERE fone_calendar.date IS NOT NULL;
""")

# Commit the changes to the database
conn.commit()

In [ ]:
# Add the Season_Phase column to fone_calendar if it doesn't already exist
#cursor.execute("""
#    ALTER TABLE fone_calendar ADD COLUMN Season_Phase TEXT;
#""")

# Update the Season_Phase column based on the month of the date
cursor.execute("""
    UPDATE fone_calendar
    SET Season_Phase = 
        CASE 
            WHEN strftime('%m', date) IN ('01','02','03', '04','05') THEN 'Early_Season'
            WHEN strftime('%m', date) IN ('06', '07', '08') THEN 'Mid_Season'
            WHEN strftime('%m', date) IN ('09','10', '11', '12') THEN 'Late_Season'
        END
    WHERE date IS NOT NULL;
""")

# Commit the changes to the database
conn.commit()

In [27]:
# Add the outbound_same_continent column to fone_calendar if it doesn't already exist
cursor.execute("""
    ALTER TABLE fone_calendar ADD COLUMN outbound_same_continent INTEGER;
""")

# Update the outbound_same_continent column based on the continent comparison
cursor.execute("""
    UPDATE fone_calendar
    SET outbound_same_continent = 
        CASE 
            WHEN (
                SELECT fg1.continent
                FROM fone_geography fg1
                WHERE fg1.id = fone_calendar.geo_id
            ) = (
                SELECT fg2.continent
                FROM travel_logistic tl
                JOIN fone_geography fg2 ON tl.to_id = fg2.id
                WHERE tl.id = fone_calendar.outbound_route
            ) THEN 1
            ELSE 0
        END
    WHERE geo_id IS NOT NULL AND outbound_route IS NOT NULL;
""")

# Commit the changes to the database
conn.commit()

In [ ]:
import pandas as pd
import sqlite3
# Ensure the database connection is open
if 'conn' not in globals() or conn is None:
    conn = sqlite3.connect('planet_fone.db')

# Read the fone_calendar table into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM fone_calendar", conn)

# Fill nulls in outbound_same_continent with 0
df['outbound_same_continent'] = df['outbound_same_continent'].fillna(0).astype(int)

# Calculate consecutive_races_in_region as a running sum of outbound_same_continent
df['consecutive_races_in_region'] = (
    df['outbound_same_continent']
    .groupby((df['outbound_same_continent'] == 0).cumsum())
    .cumsum()
)


# Write the updated DataFrame back to the database
df.to_sql('fone_calendar', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()


DatabaseError: Execution failed on sql 'SELECT * FROM fone_calendar': no such table: fone_calendar

In [8]:
# Drop the third-to-last column
df = df.drop(columns=['Consecutive_Races_In_Region'])

# Write the DataFrame to the fone_calendar table in the database
df.to_sql('fone_calendar', conn, if_exists='replace', index=False)

# Commit the changes to ensure they are saved
conn.commit()


In [ ]:
# Make a copy of the original DataFrame for safety
df_original = df.copy()

# Replace the running sum in consecutive_races_in_region with the max of each reset
df['consecutive_races_in_region'] = (
    df.groupby((df['outbound_same_continent'] == 0).cumsum())['consecutive_races_in_region']
    .transform('max')
)



,id,race_name,country,circuit,year,date,geo_id,inbound_route,outbound_route,outbound_year_emissions_t,outbound_effort_score,yearly_outbound_effort_score,leg_emissions,Time_Gap_Days,Season_Phase,outbound_same_continent,consecutive_races_in_region
0,1,Australian Grand Prix,Australia,Melbourne,2000,2000-03-12,6,NaN,239.0,NaN,NaN,NaN,NaN,14.0,Early_Season,0,0
1,2,Brazilian Grand Prix,Brazil,São Paulo,2000,2000-03-26,4,239.0,156.0,NaN,NaN,NaN,NaN,14.0,Early_Season,0,4
2,3,San Marino Grand Prix,Italy,Imola,2000,2000-04-09,16,156.0,706.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4
3,4,British Grand Prix,UK,Silverstone,2000,2000-04-23,1,706.0,9.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4
4,5,Spanish Grand Prix,Spain,Montmeló,2000,2000-05-07,10,9.0,458.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4
5,6,European Grand Prix,Germany,Nürburg,2000,2000-05-21,36,458.0,1677.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4
6,7,Monaco Grand Prix,Monaco,Monte-Carlo,2000,2000-06-04,32,1677.0,1469.0,NaN,NaN,NaN,NaN,14.0,Mid_Season,0,0


In [15]:
# Set consecutive_races_in_region to 0 where outbound_same_continent is 0
df.loc[df['outbound_same_continent'] == 0, 'consecutive_races_in_region'] = 0

In [16]:
df_original = df.copy()
df

,id,race_name,country,circuit,year,date,geo_id,inbound_route,outbound_route,outbound_year_emissions_t,outbound_effort_score,yearly_outbound_effort_score,leg_emissions,Time_Gap_Days,Season_Phase,outbound_same_continent,consecutive_races_in_region
0,1,Australian Grand Prix,Australia,Melbourne,2000,2000-03-12,6,NaN,239.0,NaN,NaN,NaN,NaN,14.0,Early_Season,0,0
1,2,Brazilian Grand Prix,Brazil,São Paulo,2000,2000-03-26,4,239.0,156.0,NaN,NaN,NaN,NaN,14.0,Early_Season,0,0
2,3,San Marino Grand Prix,Italy,Imola,2000,2000-04-09,16,156.0,706.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4
3,4,British Grand Prix,UK,Silverstone,2000,2000-04-23,1,706.0,9.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4
4,5,Spanish Grand Prix,Spain,Montmeló,2000,2000-05-07,10,9.0,458.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,507,Mexico City Grand Prix,Mexico,Mexico City,2025,2025-10-26,17,674.0,756.0,NaN,NaN,NaN,NaN,14.0,Late_Season,0,0
507,508,São Paulo Grand Prix,Brazil,São Paulo,2025,2025-11-09,4,756.0,173.0,NaN,NaN,NaN,NaN,13.0,Late_Season,0,0
508,509,Las Vegas Grand Prix,United States,Las Vegas,2025,2025-11-22,33,173.0,1532.0,NaN,NaN,NaN,NaN,8.0,Late_Season,0,0
509,510,Qatar Grand Prix,Qatar,Lusail,2025,2025-11-30,28,1532.0,1305.0,NaN,NaN,NaN,NaN,7.0,Late_Season,1,1


In [17]:
# Ensure the DataFrame is sorted by date
df = df.sort_values(by='date').reset_index(drop=True)

# Create the stint_position column based on the conditions
df['stint_position'] = 'Single'  # Default value

# Iterate through the DataFrame to assign values based on conditions
for i in range(len(df)):
    if df.loc[i, 'outbound_same_continent'] == 1:
        if i == 0 or df.loc[i - 1, 'outbound_same_continent'] == 0:
            df.loc[i, 'stint_position'] = 'Start'
        elif i == len(df) - 1 or df.loc[i + 1, 'outbound_same_continent'] == 0:
            df.loc[i, 'stint_position'] = 'End'
        else:
            df.loc[i, 'stint_position'] = 'Middle'

# Verify the result
df[['date', 'outbound_same_continent', 'stint_position']]

,date,outbound_same_continent,stint_position
0,2000-03-12,0,Single
1,2000-03-26,0,Single
2,2000-04-09,1,Start
3,2000-04-23,1,Middle
4,2000-05-07,1,Middle
...,...,...,...
506,2025-10-26,0,Single
507,2025-11-09,0,Single
508,2025-11-22,0,Single
509,2025-11-30,1,Start


In [19]:
import pandas as pd

# Set pandas options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [36]:
# Rename the column 'Time_Gap_Days' to 'Gap_Next_Days'
df.rename(columns={'Time_Gap_Days': 'Gap_Next_Days'}, inplace=True)

# Calculate the 'Back_To_Back' flag
df['time_to_arrive_here'] = df['Gap_Next_Days'].shift(1).apply(
    lambda x: '1W' if x <= 9 else ('2W' if x <= 16 else '3W+')
)
df['involved_in_back_to_back'] = df['Back_To_Back'] | df['Back_To_Back'].shift(-1, fill_value=0)
df['involved_in_triple_header'] = (
    (df['involved_in_back_to_back'] == 1) & 
    ((df['Back_To_Back'].shift(-1, fill_value=0) == 1) & 
     (df['Back_To_Back'].shift(-2, fill_value=0) == 1) |
     (df['Back_To_Back'].shift(-1, fill_value=0) == 1) & 
     (df['Back_To_Back'].shift(0, fill_value=0) == 1) |
     (df['Back_To_Back'].shift(0, fill_value=0) == 1) & 
     (df['Back_To_Back'].shift(1, fill_value=0) == 1))
).astype(int)
df.tail(2)  # Display the last 10 rows of the DataFrame to verify the changes


,id,race_name,country,circuit,year,date,geo_id,inbound_route,outbound_route,outbound_year_emissions_t,outbound_effort_score,yearly_outbound_effort_score,leg_emissions,Gap_Next_Days,Season_Phase,outbound_same_continent,consecutive_races_in_region,stint_position,Back_To_Back,involved_in_back_to_back,involved_in_triple_header,time_do_arrive_here,time_to_arrive_here
509,510,Qatar Grand Prix,Qatar,Lusail,2025,2025-11-30,28,1532.0,1305.0,NaN,NaN,NaN,NaN,7.0,Late_Season,1,1,Start,1,1,1,1W,1W
510,511,Abu Dhabi Grand Prix,United Arab Emirates,Yas Island,2025,2025-12-07,37,1305.0,NaN,NaN,NaN,NaN,NaN,NaN,Late_Season,0,0,Single,1,1,1,1W,1W


In [39]:
# Drop the specified columns if they exist
df = df.drop(columns=['Back_To_Back', 'time_do_arrive_here'], errors='ignore')

In [40]:
df.head(10)  # Display the first 10 rows of the DataFrame to verify the changes

,id,race_name,country,circuit,year,date,geo_id,inbound_route,outbound_route,outbound_year_emissions_t,outbound_effort_score,yearly_outbound_effort_score,leg_emissions,Gap_Next_Days,Season_Phase,outbound_same_continent,consecutive_races_in_region,stint_position,involved_in_back_to_back,involved_in_triple_header,time_to_arrive_here
0,1,Australian Grand Prix,Australia,Melbourne,2000,2000-03-12,6,NaN,239.0,NaN,NaN,NaN,NaN,14.0,Early_Season,0,0,Single,0,0,3W+
1,2,Brazilian Grand Prix,Brazil,São Paulo,2000,2000-03-26,4,239.0,156.0,NaN,NaN,NaN,NaN,14.0,Early_Season,0,0,Single,0,0,2W
2,3,San Marino Grand Prix,Italy,Imola,2000,2000-04-09,16,156.0,706.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4,Start,0,0,2W
3,4,British Grand Prix,UK,Silverstone,2000,2000-04-23,1,706.0,9.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4,Middle,0,0,2W
4,5,Spanish Grand Prix,Spain,Montmeló,2000,2000-05-07,10,9.0,458.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4,Middle,0,0,2W
5,6,European Grand Prix,Germany,Nürburg,2000,2000-05-21,36,458.0,1677.0,NaN,NaN,NaN,NaN,14.0,Early_Season,1,4,End,0,0,2W
6,7,Monaco Grand Prix,Monaco,Monte-Carlo,2000,2000-06-04,32,1677.0,1469.0,NaN,NaN,NaN,NaN,14.0,Mid_Season,0,0,Single,0,0,2W
7,8,Canadian Grand Prix,Canada,Montreal,2000,2000-06-18,12,1469.0,536.0,NaN,NaN,NaN,NaN,14.0,Mid_Season,0,0,Single,0,0,2W
8,9,French Grand Prix,France,Magny Cours,2000,2000-07-02,20,536.0,904.0,NaN,NaN,NaN,NaN,14.0,Mid_Season,1,5,Start,0,0,2W
9,10,Austrian Grand Prix,Austria,Spielberg,2000,2000-07-16,11,904.0,482.0,NaN,NaN,NaN,NaN,14.0,Mid_Season,1,5,Middle,0,0,2W


In [41]:
# Write the DataFrame to the fone_calendar table in the database
df.to_sql('fone_calendar', conn, if_exists='replace', index=False)

# Commit the changes to ensure they are saved
conn.commit()

In [56]:
import pandas as pd
import sqlite3

# Ensure the database connection is open
if 'conn' not in globals() or conn is None:
    conn = sqlite3.connect('planet_fone.db')

# Define the query
query = """
select
    fc.outbound_route         as id
    , fc.year
    , fg_from.circuit_x         as from_circuit
    , fg_to.circuit_x         as to_circuit
    , fg_from.latitude         as "from_lat"
    , fg_from.longitude         as "from_lon"
    , fg_to.latitude             as "to_lat"
    , fg_to.longitude             as "to_lon"
    , abs(fg_from.latitude-fg_to.latitude)as "delta_lat"
    , abs(fg_from.longitude-fg_to.longitude) as  "delta_lon"
    , fc.consecutive_races_in_region
    , fc.stint_position
    , fc.involved_in_back_to_back
    , fc.involved_in_triple_header
    , case when
       fg_from.city_x
        in ()
        then 1
        else 0
        end                    as "from_is_island"
    , case when
       fg_to.city_x
        in ()
        then 1
        else 0
        end                    as "to_is_island"
    , fc.Gap_Next_Days
    , fc.leg_emissions             as "estimated_emissions_kgCO2e"
    , tl.distance_km             as "air_distance_km"
    , tl.truck_distance_km
    , case when 
        tl.truck_distance_km 
        is not null 
        then 1 
        else 0 
        end                     as "truck_feasible"
    , tl.truck_viable            as "truck_mode_assigned"
    , case when
        fg_from.continent=fg_to.continent
        then 0
        else 1
        end                    as "is_intercontinental"
from
    fone_calendar fc
left join
    travel_logistic tl
    on fc.outbound_route = tl.id
left join
    fone_geography fg_from
    on tl.from_id = fg_from.id
left join
    fone_geography fg_to
    on tl.to_id = fg_to.id
where
    1=1
    and fc.leg_emissions is not null
"""

# Execute the query and load the result into a DataFrame
df_query_result = pd.read_sql_query(query, conn)

# Copy the result to the clipboard
df_query_result.to_clipboard(index=False)

print("Query result copied to clipboard.")

Query result copied to clipboard.


In [57]:
df_query_result.head(10)  # Display the first 10 rows of the DataFrame to verify the changes

,id,year,from_circuit,to_circuit,from_lat,from_lon,to_lat,to_lon,delta_lat,delta_lon,consecutive_races_in_region,stint_position,involved_in_back_to_back,involved_in_triple_header,from_is_island,to_is_island,Gap_Next_Days,estimated_emissions_kgCO2e,air_distance_km,truck_distance_km,truck_feasible,truck_mode_assigned,is_intercontinental
0,240.0,2019,Melbourne,Sakhir,-37.8497,144.9680,26.0325,50.5106,63.8822,94.4574,0,Single,0,0,0,0,14.0,13741.515183,12112.617740,NaN,0,0,1
1,195.0,2019,Sakhir,Shanghai,26.0325,50.5106,31.3303,121.2246,5.2978,70.7140,2,Start,0,0,0,0,14.0,7719.891120,6804.787455,NaN,0,0,0
2,346.0,2019,Shanghai,Baku,31.3303,121.2246,40.3725,49.8533,9.0422,71.3713,2,End,0,0,0,0,14.0,7189.026673,6336.850839,NaN,0,0,0
3,809.0,2019,Baku,Catalunya,40.3725,49.8533,41.5700,2.2611,1.1975,47.5922,0,Single,0,0,0,0,14.0,4477.219229,3946.496754,NaN,0,0,1
4,454.0,2019,Catalunya,Monte Carlo,41.5700,2.2611,43.7347,7.4206,2.1647,5.1595,1,Start,0,0,0,0,14.0,67.539188,485.644843,661.48,1,1,0
5,1469.0,2019,Monte Carlo,Montreal,43.7347,7.4206,45.5000,-73.5228,1.7653,80.9434,0,Single,0,0,0,0,14.0,6944.978740,6121.731961,NaN,0,0,1
6,543.0,2019,Montreal,Le Castellet,45.5000,-73.5228,43.2500,5.7917,2.2500,79.3145,0,Single,0,0,0,0,14.0,6851.405914,6039.251110,NaN,0,0,1
7,1233.0,2019,Le Castellet,Spielberg,43.2500,5.7917,47.2196,14.7649,3.9696,8.9732,6,Start,1,0,0,0,7.0,116.578308,828.988270,1141.77,1,1,0
8,471.0,2019,Spielberg,Silverstone,47.2196,14.7649,52.0786,-1.0169,4.8590,15.7818,6,Middle,1,0,0,0,14.0,162.037691,1254.656219,1587.00,1,1,0
9,12.0,2019,Silverstone,Hockenheim,52.0786,-1.0169,49.3249,8.5694,2.7537,9.5863,6,Middle,0,0,0,0,14.0,100.265288,740.181551,982.00,1,1,0


In [58]:
import pandas as pd
import numpy as np # For potential NaN handling if needed later

# --- Load your dataset first ---
try:
    df_f1_legs = df_query_result

    # --- Identify the categorical columns to encode ---
    categorical_cols = ['stint_position']
    # Note: 'from_circuit', 'to_circuit' are also objects/categorical,
    # but typically not used as features directly. If you needed them,
    # you would add them to this list.

    print(f"Original DataFrame shape: {df_f1_legs.shape}")
    print(f"Original columns: {df_f1_legs.columns.tolist()}")
    print("\nUnique values in 'stint_position':", df_f1_legs['stint_position'].unique())


    # --- Apply One-Hot Encoding ---
    df_encoded = pd.get_dummies(df_f1_legs,
                                columns=categorical_cols,
                                prefix=categorical_cols, # Optional: Adds prefix to new columns
                                prefix_sep='_',         # Optional: Separator
                                dummy_na=False,         # Set True if you want a column for NaN values
                                drop_first=False)       # Set True to drop first category (avoid dummy trap)

    print(f"\nEncoded DataFrame shape: {df_encoded.shape}")
    print(f"Encoded columns: {df_encoded.columns.tolist()}")

    print("\nEncoded DataFrame Head:")
    # Displaying relevant columns to see the transformation
    original_cols_to_show = ['id', 'year', 'from_circuit', 'to_circuit']
    encoded_cols_to_show = [col for col in df_encoded.columns if col.startswith(tuple(categorical_cols))]
    print(df_encoded[original_cols_to_show + encoded_cols_to_show].head())

except Exception as e:
    print(f"An error occurred: {e}")

Original DataFrame shape: (42, 23)
Original columns: ['id', 'year', 'from_circuit', 'to_circuit', 'from_lat', 'from_lon', 'to_lat', 'to_lon', 'delta_lat', 'delta_lon', 'consecutive_races_in_region', 'stint_position', 'involved_in_back_to_back', 'involved_in_triple_header', 'from_is_island', 'to_is_island', 'Gap_Next_Days', 'estimated_emissions_kgCO2e', 'air_distance_km', 'truck_distance_km', 'truck_feasible', 'truck_mode_assigned', 'is_intercontinental']

Unique values in 'stint_position': ['Single' 'Start' 'End' 'Middle']

Encoded DataFrame shape: (42, 26)
Encoded columns: ['id', 'year', 'from_circuit', 'to_circuit', 'from_lat', 'from_lon', 'to_lat', 'to_lon', 'delta_lat', 'delta_lon', 'consecutive_races_in_region', 'involved_in_back_to_back', 'involved_in_triple_header', 'from_is_island', 'to_is_island', 'Gap_Next_Days', 'estimated_emissions_kgCO2e', 'air_distance_km', 'truck_distance_km', 'truck_feasible', 'truck_mode_assigned', 'is_intercontinental', 'stint_position_End', 'stint_po

In [64]:
df_encoded['truck_distance_km'] = df_encoded['truck_distance_km'].fillna(0)

In [65]:
df_encoded.head(10)  # Display the first 10 rows of the DataFrame to verify the changes
# Write the DataFrame to the fone_calendar table in the database

,id,year,from_circuit,to_circuit,from_lat,from_lon,to_lat,to_lon,delta_lat,delta_lon,consecutive_races_in_region,involved_in_back_to_back,involved_in_triple_header,from_is_island,to_is_island,Gap_Next_Days,estimated_emissions_kgCO2e,air_distance_km,truck_distance_km,truck_feasible,truck_mode_assigned,is_intercontinental,stint_position_End,stint_position_Middle,stint_position_Single,stint_position_Start
0,240.0,2019,Melbourne,Sakhir,-37.8497,144.9680,26.0325,50.5106,63.8822,94.4574,0,0,0,0,0,14.0,13741.515183,12112.617740,0.00,0,0,1,False,False,True,False
1,195.0,2019,Sakhir,Shanghai,26.0325,50.5106,31.3303,121.2246,5.2978,70.7140,2,0,0,0,0,14.0,7719.891120,6804.787455,0.00,0,0,0,False,False,False,True
2,346.0,2019,Shanghai,Baku,31.3303,121.2246,40.3725,49.8533,9.0422,71.3713,2,0,0,0,0,14.0,7189.026673,6336.850839,0.00,0,0,0,True,False,False,False
3,809.0,2019,Baku,Catalunya,40.3725,49.8533,41.5700,2.2611,1.1975,47.5922,0,0,0,0,0,14.0,4477.219229,3946.496754,0.00,0,0,1,False,False,True,False
4,454.0,2019,Catalunya,Monte Carlo,41.5700,2.2611,43.7347,7.4206,2.1647,5.1595,1,0,0,0,0,14.0,67.539188,485.644843,661.48,1,1,0,False,False,False,True
5,1469.0,2019,Monte Carlo,Montreal,43.7347,7.4206,45.5000,-73.5228,1.7653,80.9434,0,0,0,0,0,14.0,6944.978740,6121.731961,0.00,0,0,1,False,False,True,False
6,543.0,2019,Montreal,Le Castellet,45.5000,-73.5228,43.2500,5.7917,2.2500,79.3145,0,0,0,0,0,14.0,6851.405914,6039.251110,0.00,0,0,1,False,False,True,False
7,1233.0,2019,Le Castellet,Spielberg,43.2500,5.7917,47.2196,14.7649,3.9696,8.9732,6,1,0,0,0,7.0,116.578308,828.988270,1141.77,1,1,0,False,False,False,True
8,471.0,2019,Spielberg,Silverstone,47.2196,14.7649,52.0786,-1.0169,4.8590,15.7818,6,1,0,0,0,14.0,162.037691,1254.656219,1587.00,1,1,0,False,True,False,False
9,12.0,2019,Silverstone,Hockenheim,52.0786,-1.0169,49.3249,8.5694,2.7537,9.5863,6,0,0,0,0,14.0,100.265288,740.181551,982.00,1,1,0,False,True,False,False


In [66]:
df_encoded.to_sql('training_regression_calendar', conn, if_exists='replace', index=False)
df_encoded.to_csv('training_regression_calendar.csv', index=False)

adding codes

In [11]:
# Ensure the database connection is open
conn = sqlite3.connect('planet_fone.db')




# Update the new column with the generated 6-letter codes, skipping spaces in substrings
cursor.execute("""
    UPDATE fone_geography
    SET code_6 = UPPER(
        SUBSTR(REPLACE(country_x, ' ', ''), 1, 3) || 
        SUBSTR(REPLACE(circuit_x, ' ', ''), 1, 3)
    )
    WHERE circuit_x IS NOT NULL AND country_x IS NOT NULL;
""")

# Commit the changes to the database
conn.commit()
conn.close()

In [2]:
import pandas as pd
import sqlite3

# Ensure the database connection is open
conn = sqlite3.connect('planet_fone.db')

# Load the fone_geography table into a pandas DataFrame
df_geography = pd.read_sql_query("SELECT * FROM fone_geography", conn)

# Update codes for United States
df_geography.loc[df_geography['country_x'] == 'United States', 'code_6'] = (
    df_geography.loc[df_geography['country_x'] == 'United States', 'code_6'].str.replace('UNI', 'USA', regex=False)
)

# Update codes for United Kingdom
df_geography.loc[df_geography['country_x'] == 'United Kingdom', 'code_6'] = (
    df_geography.loc[df_geography['country_x'] == 'United Kingdom', 'code_6'].str.replace('UNI', 'UKG', regex=False)
)

# Write the updated DataFrame back to the database
df_geography.to_sql('fone_geography', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()


In [5]:
# Ensure the database connection is open
conn = sqlite3.connect('planet_fone.db')

# Load the fone_calendar table into a pandas DataFrame
df_calendar = pd.read_sql_query("SELECT * FROM fone_calendar", conn)

# Merge fone_calendar with fone_geography to get city information
df_merged = pd.merge(
    df_calendar,
    df_geography[['id', 'city_x']],
    left_on='geo_id',
    right_on='id',
    suffixes=('', '_geo')
)
# Filter out records where race_name doesn't end with "Grand Prix"
df_merged = df_merged[df_merged['race_name'].str.endswith("Grand Prix")]
# Filter the first Grand Prix of each year
df_first_gp = df_merged.sort_values(by='date').groupby('year').first()

# Calculate the probability of each city being the first Grand Prix
city_first_gp_counts = df_first_gp['city_x'].value_counts()
city_first_gp_probabilities = city_first_gp_counts / df_first_gp.shape[0]

# Add the probabilities to the fone_geography DataFrame
df_geography['first_gp_probability'] = df_geography['city_x'].map(city_first_gp_probabilities).fillna(0)

# Write the updated fone_geography DataFrame back to the database
df_geography.to_sql('fone_geography', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()

# Filter the last Grand Prix of each year
df_last_gp = df_merged.sort_values(by='date').groupby('year').last()

# Calculate the probability of each city being the last Grand Prix
city_last_gp_counts = df_last_gp['city_x'].value_counts()
city_last_gp_probabilities = city_last_gp_counts / df_last_gp.shape[0]

# Add the probabilities to the fone_geography DataFrame
df_geography['last_gp_probability'] = df_geography['city_x'].map(city_last_gp_probabilities).fillna(0)

# Write the updated fone_geography DataFrame back to the database
df_geography.to_sql('fone_geography', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()

In [7]:
# Ensure the database connection is open
if 'conn' not in globals() or conn is None:
    conn = sqlite3.connect('planet_fone.db')

# Load the travel_logistic table into a pandas DataFrame
df_travel_logistic = pd.read_sql_query("SELECT * FROM travel_logistic", conn)

# Load the fone_geography table into a pandas DataFrame
df_fone_geography = pd.read_sql_query("SELECT id, code_6 FROM fone_geography", conn)

# Merge travel_logistic with fone_geography to get the code_6 for from_id
df_travel_logistic = df_travel_logistic.merge(
    df_fone_geography,
    left_on='from_id',
    right_on='id',
    how='left',
    suffixes=('', '_from')
)

# Merge travel_logistic with fone_geography to get the code_6 for to_id
df_travel_logistic = df_travel_logistic.merge(
    df_fone_geography,
    left_on='to_id',
    right_on='id',
    how='left',
    suffixes=('', '_to')
)

# Create the new 'codes' column by concatenating the code_6 values
df_travel_logistic['codes'] = df_travel_logistic['code_6'] + '-' + df_travel_logistic['code_6_to']

# Drop unnecessary columns
df_travel_logistic = df_travel_logistic.drop(columns=['id_from', 'id_to', 'code_6', 'code_6_to'])

# Write the updated DataFrame back to the database
df_travel_logistic.to_sql('travel_logistic', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()

Enhance training dataset with new total emission estimations

In [2]:
estimated_total_emissions = [
  {
    "Year": 2000,
    "Number of Races": 17,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 95500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "Baseline assumptions"
  },
  {
    "Year": 2001,
    "Number of Races": 17,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 95500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2002,
    "Number of Races": 17,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 95500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2003,
    "Number of Races": 16,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 90000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "Fewer races"
  },
  {
    "Year": 2004,
    "Number of Races": 18,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 101000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "DHL becomes logistics partner"
  },
  {
    "Year": 2005,
    "Number of Races": 19,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 106500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "Increase in races"
  },
  {
    "Year": 2006,
    "Number of Races": 18,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 101000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2007,
    "Number of Races": 17,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 95500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2008,
    "Number of Races": 18,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 101000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2009,
    "Number of Races": 17,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 95500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2010,
    "Number of Races": 19,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 106500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2011,
    "Number of Races": 19,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 106500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2012,
    "Number of Races": 20,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 112000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2013,
    "Number of Races": 19,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 106500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2014,
    "Number of Races": 19,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 106500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2015,
    "Number of Races": 19,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 106500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2016,
    "Number of Races": 21,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 117500,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "Calendar expansion"
  },
  {
    "Year": 2017,
    "Number of Races": 20,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 112000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2018,
    "Number of Races": 21,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 115448,
    "Data Source": "Official (calc.)",
    "Key Logistical Changes/Notes": "F1 Baseline Year"
  },
  {
    "Year": 2019,
    "Number of Races": 21,
    "Predominant Inter-continental Aircraft": "B747F",
    "Calculated/Estimated Logistics CO2e (t)": 115448,
    "Data Source": "Official (calc.)",
    "Key Logistical Changes/Notes": ""
  },
  {
    "Year": 2020,
    "Number of Races": 17,
    "Predominant Inter-continental Aircraft": "B777F",
    "Calculated/Estimated Logistics CO2e (t)": 65000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "COVID-19 impacted calendar (reduced fly-aways, B777 transition starts)"
  },
  {
    "Year": 2021,
    "Number of Races": 22,
    "Predominant Inter-continental Aircraft": "B777F",
    "Calculated/Estimated Logistics CO2e (t)": 105000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "COVID-19 impacted, more fly-aways than 2020, B777F primary air freight"
  },
  {
    "Year": 2022,
    "Number of Races": 22,
    "Predominant Inter-continental Aircraft": "B777F",
    "Calculated/Estimated Logistics CO2e (t)": 109285,
    "Data Source": "Official (calc.)",
    "Key Logistical Changes/Notes": "Two more fly-aways vs 2018"
  },
  {
    "Year": 2023,
    "Number of Races": 22,
    "Predominant Inter-continental Aircraft": "B777F",
    "Calculated/Estimated Logistics CO2e (t)": 108000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "HVO trucks for European leg (9 races), calendar regionalization efforts begin to influence"
  },
  {
    "Year": 2024,
    "Number of Races": 24,
    "Predominant Inter-continental Aircraft": "B777F",
    "Calculated/Estimated Logistics CO2e (t)": 115000,
    "Data Source": "Estimated",
    "Key Logistical Changes/Notes": "SAF use initiated, further calendar regionalization, record number of races"
  }
]


In [3]:
import pandas as pd
import sys
import os
# Get the absolute path of the directory where the notebook is currently running
# This usually is /path/to/app/folder_notebook
notebook_dir = os.getcwd()

# Get the absolute path of the 'app' directory (parent of folder_notebook)
main_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
db_path = os.path.join(main_dir, 'data', 'planet_fone.db')
if main_dir not in sys.path:
    sys.path.append(main_dir)
    
from utils import sql

In [4]:
fc = sql.get_table('fone_calendar', db_path)

2025-05-25 09:21:28,004 - INFO - get_table - Executing function: get_table
2025-05-25 09:21:28,010 - INFO - get_table - Completed function: get_table


In [5]:

# Convert estimated_total_emissions to DataFrame
df_emissions = pd.DataFrame(estimated_total_emissions)

# Merge the emissions data into fc on year
fc = fc.merge(
    df_emissions[['Year', 'Calculated/Estimated Logistics CO2e (t)']],
    left_on='year',
    right_on='Year',
    how='left'
)

# Update outbound_year_emissions_t in fc
fc['outbound_year_emissions_t'] = fc['Calculated/Estimated Logistics CO2e (t)']

# Drop the helper columns
fc = fc.drop(columns=['Year', 'Calculated/Estimated Logistics CO2e (t)'])


fix to assess real working seasons
*1. remove 2020
*2. remove test races

In [6]:
fc = fc[fc['year']!= 2020]  # Exclude 2020 as it is not relevant for the emissions data
fc = fc[fc['year']!= 2021]  # Exclude 2021 as it is not relevant for the emissions data
fc = fc[fc['race_name'].str.endswith("Grand Prix")]  # Filter to keep only Grand Prix races

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
fc.tail(200)  # Display the first 10 rows of the DataFrame to verify the changes

,id,race_name,country,circuit,year,date,geo_id,inbound_route,outbound_route,outbound_year_emissions_t,outbound_effort_score,yearly_outbound_effort_score,leg_emissions,Gap_Next_Days,Season_Phase,outbound_same_continent,consecutive_races_in_region,stint_position,involved_in_back_to_back,involved_in_triple_header,time_to_arrive_here,n_efforts
240,255,Chinese Grand Prix,China,Shanghai,2014,2014-04-20,8,195.0,338.0,106500.0,9779.945565,96244.042333,10822.116127,21.0,Early_Season,0,0,Single,0,0,2W,18
241,256,Spanish Grand Prix,Spain,Montmeló,2014,2014-05-11,10,338.0,454.0,106500.0,59.533200,96244.042333,65.877177,14.0,Early_Season,1,1,Start,0,0,3W+,18
242,257,Monaco Grand Prix,Monaco,Monte-Carlo,2014,2014-05-25,32,454.0,1469.0,106500.0,6121.731961,96244.042333,6774.075964,14.0,Early_Season,0,0,Single,0,0,2W,18
243,258,Canadian Grand Prix,Canada,Montreal,2014,2014-06-08,12,1469.0,528.0,106500.0,6390.504231,96244.042333,7071.489145,14.0,Mid_Season,0,0,Single,0,0,2W,18
244,259,Austrian Grand Prix,Austria,Spielberg,2014,2014-06-22,11,528.0,471.0,106500.0,142.830000,96244.042333,158.050251,14.0,Mid_Season,1,5,Start,0,0,2W,18
245,260,British Grand Prix,UK,Silverstone,2014,2014-07-06,1,471.0,12.0,106500.0,88.380000,96244.042333,97.797950,14.0,Mid_Season,1,5,Middle,0,0,2W,18
246,261,German Grand Prix,Germany,Hockenheim,2014,2014-07-20,13,12.0,566.0,106500.0,91.530000,96244.042333,101.283620,7.0,Mid_Season,1,5,Middle,1,0,2W,18
247,262,Hungarian Grand Prix,Hungary,Budapest,2014,2014-07-27,2,566.0,67.0,106500.0,115.323300,96244.042333,127.612382,28.0,Mid_Season,1,5,Middle,1,0,1W,18
248,263,Belgian Grand Prix,Belgium,Spa,2014,2014-08-24,21,67.0,943.0,106500.0,71.404200,96244.042333,79.013174,14.0,Mid_Season,1,5,End,0,0,3W+,18
249,264,Italian Grand Prix,Italy,Monza,2014,2014-09-07,3,943.0,107.0,106500.0,10261.009679,96244.042333,11354.443395,14.0,Late_Season,0,0,Single,0,0,2W,18


In [8]:
tl = sql.get_table('travel_logistic', db_path)

2025-05-25 09:21:34,257 - INFO - get_table - Executing function: get_table
2025-05-25 09:21:34,266 - INFO - get_table - Completed function: get_table


In [9]:
# Update fc['effort_score'] with values from tl where tl.id matches fc['outbound_route']
fc = fc.merge(
    tl[['id', 'effort_score']],
    left_on='outbound_route',
    right_on='id',
    how='left',
    suffixes=('', '_tl')
)

# Drop records where outbound_route did not find a match in tl (i.e., effort_score is null)
fc = fc[fc['effort_score'].notnull()].reset_index(drop=True)

# Only update where outbound_route is not null
fc.loc[fc['outbound_route'].notnull(), 'outbound_effort_score'] = fc.loc[fc['outbound_route'].notnull(), 'effort_score']

# Drop the helper columns
fc = fc.drop(columns=['id_tl', 'effort_score'])

In [10]:
fc[fc['outbound_effort_score'].isnull()]  # Check if there are still any null values in outbound_effort_score

,id,race_name,country,circuit,year,date,geo_id,inbound_route,outbound_route,outbound_year_emissions_t,outbound_effort_score,yearly_outbound_effort_score,leg_emissions,Gap_Next_Days,Season_Phase,outbound_same_continent,consecutive_races_in_region,stint_position,involved_in_back_to_back,involved_in_triple_header,time_to_arrive_here,n_efforts


In [11]:
fc['yearly_outbound_effort_score'] = fc.groupby('year')['outbound_effort_score'].transform('sum')
fc['n_efforts'] = fc.groupby('year')['outbound_effort_score'].transform('count')


In [12]:
fc['leg_emissions'] = (
    fc['outbound_year_emissions_t'] * (fc['outbound_effort_score'] / fc['yearly_outbound_effort_score'])
)

In [14]:
fc[fc['outbound_route']==156]

,id,race_name,country,circuit,year,date,geo_id,inbound_route,outbound_route,outbound_year_emissions_t,outbound_effort_score,yearly_outbound_effort_score,leg_emissions,Gap_Next_Days,Season_Phase,outbound_same_continent,consecutive_races_in_region,stint_position,involved_in_back_to_back,involved_in_triple_header,time_to_arrive_here,n_efforts
1,2,Brazilian Grand Prix,Brazil,São Paulo,2000,2000-03-26,4,239.0,156.0,95500.0,9612.464691,58454.434190,15704.375394,14.0,Early_Season,0,0,Single,0,0,2W,16
18,20,Brazilian Grand Prix,Brazil,São Paulo,2001,2001-04-01,4,380.0,156.0,95500.0,9612.464691,62505.004158,14686.670137,14.0,Early_Season,0,0,Single,0,0,2W,16
34,37,Brazilian Grand Prix,Brazil,São Paulo,2002,2002-03-31,4,380.0,156.0,95500.0,9612.464691,62483.691258,14691.679693,14.0,Early_Season,0,0,Single,0,0,2W,16
50,54,Brazilian Grand Prix,Brazil,São Paulo,2003,2003-04-06,4,380.0,156.0,90000.0,9612.464691,62409.619458,13861.994829,14.0,Early_Season,0,0,Single,0,0,2W,15


In [2]:
import pandas as pd
import sys
import os
# Get the absolute path of the directory where the notebook is currently running
# This usually is /path/to/app/folder_notebook
notebook_dir = os.getcwd()

# Get the absolute path of the 'app' directory (parent of folder_notebook)
main_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
db_path = os.path.join(main_dir, 'data', 'planet_fone.db')
data_path = os.path.join(main_dir, 'data')
if main_dir not in sys.path:
    sys.path.append(main_dir)
    
from utils import sql

In [3]:
import pandas as pd
import sqlite3
import os

# Load the CSV file into a DataFrame
csv_path = os.path.join(data_path, 'fone_calendar.csv')
df = pd.read_csv(csv_path)

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Replace the fone_calendar table with the DataFrame
df.to_sql('fone_calendar', conn, if_exists='replace', index=False)

# Commit and close the connection
conn.commit()
conn.close()

4